# Create Sentinel2Dataset

In [21]:
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
import cv2
import numpy as np
import pandas as pd
import torch
import os
import natsort
import glob
from PIL import Image
import natsort
import cv2

In [22]:
DATASET_VERSION = "V2"
TRAIN_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/train/"
VAL_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/val/"
TEST_DIR = f"/mnt/disk/dataset/sentinel-ai-processor/{DATASET_VERSION}/test/"



def prepare_paths(path_dir):
    
    
    df_input = pd.read_csv(f"{path_dir}/input.csv")
    df_output = pd.read_csv(f"{path_dir}/target.csv")

    df_input["path"] = df_input["Name"].apply(lambda x: os.path.join(path_dir, "input", os.path.basename(x).replace(".SAFE","")))
    df_output["path"] = df_output["Name"].apply(lambda x: os.path.join(path_dir, "input", os.path.basename(x).replace(".SAFE","")))
    
    return df_input, df_output


df_input, df_output =  prepare_paths(TEST_DIR)

In [28]:
def normalize(band, lower_percent=2, upper_percent=98):
    """
    Apply percentile stretching to enhance contrast, only considering valid pixels.
    
    Args:
        band: Input image band as numpy array
        lower_percent: Lower percentile boundary (default 2%)
        upper_percent: Upper percentile boundary (default 98%)
        
    Returns:
        Normalized band with values in [0, 1]
    """
    # Create mask for valid pixels
    valid_mask = (band > 0)
    
    # If no valid pixels, return zeros
    if not np.any(valid_mask):
        return np.zeros_like(band, dtype=np.float32)
    
    # Extract valid pixels for percentile calculation
    valid_pixels = band[valid_mask]
    # Calculate percentiles based only on valid pixels
    lower = np.percentile(valid_pixels, lower_percent)
    upper = np.percentile(valid_pixels, upper_percent)
    
    # Create a copy to avoid modifying the original
    result = band.copy().astype(np.float32)
    
    # Apply stretching only to valid pixels
    result[valid_mask] = np.clip((band[valid_mask] - lower) / (upper - lower), 0, 1)
    
    # Set invalid pixels to 0
    result[~valid_mask] = 0
    
    return result


def read_images(product_paths):

    images = []

    for path in product_paths:

        data = Image.open(path)
        data = np.array(data)
        data = normalize(data)
        
        images.append(data)
        
    
    
    images = np.dstack(images)
    return images

In [ ]:
class Sentinel2Dataset(Dataset):
    def __init__(self, df_x, df_y,
                 train,
                 augmentation,
                 img_size):

        self.df_x = df_x
        self.df_y = df_y
        self.train = train
        self.augmentation = augmentation
        self.img_size = img_size
        # self.transform = get_transforms(train=self.train,
        #                                 augmentation=self.augmentation)

    def __getitem__(self, index):
        # Load images
        x_paths = natsort.natsorted(glob.glob(os.path.join(self.df_x["path"][index], "*.png"), recursive=False))
        x_data = read_images(x_paths)
        x_data = cv2.resize(x_data, (self.img_size, self.img_size), interpolation=cv2.INTER_AREA)
        x_data = torch.from_numpy(x_data).float()
        x_data = torch.permute(x_data, (2, 0, 1))  # HWC to CHW

        y_paths = natsort.natsorted(glob.glob(os.path.join(self.df_x["path"][index], "*.png"), recursive=False))
        y_data = read_images(y_paths)
        y_data = cv2.resize(y_data, (self.img_size, self.img_size), interpolation=cv2.INTER_AREA)
        y_data = torch.from_numpy(y_data).float()
        y_data = torch.permute(y_data, (2, 0, 1))  # HWC to CHW

        # transformed = self.transform(image=x_data, mask=y_data)
        # y_data = transformed["mask"]
        # x_data = transformed["image"]

        return x_data, y_data

    def __len__(self):
        return len(self.df_x)

In [42]:
from torch.utils.data import DataLoader

train_dataset = Sentinel2Dataset(df_x=df_input, df_y=df_output, train=True, augmentation=False, img_size=1024)
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=False,
    drop_last=True,
    num_workers=8,
)

In [43]:
for x_data, y_data in train_loader:
    
    print(x_data.shape, x_data.shape)
    

torch.Size([16, 3, 1024, 1024]) torch.Size([16, 3, 1024, 1024])
torch.Size([16, 3, 1024, 1024]) torch.Size([16, 3, 1024, 1024])
torch.Size([16, 3, 1024, 1024]) torch.Size([16, 3, 1024, 1024])
torch.Size([16, 3, 1024, 1024]) torch.Size([16, 3, 1024, 1024])
torch.Size([16, 3, 1024, 1024]) torch.Size([16, 3, 1024, 1024])
torch.Size([16, 3, 1024, 1024]) torch.Size([16, 3, 1024, 1024])
torch.Size([16, 3, 1024, 1024]) torch.Size([16, 3, 1024, 1024])
torch.Size([16, 3, 1024, 1024]) torch.Size([16, 3, 1024, 1024])


Exception ignored in: Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/ai_processor/lib/python3.12/multiprocessing/queues.py", line 259, in _feed
    reader_close()
  File "/home/ubuntu/miniconda3/envs/ai_processor/lib/python3.12/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/home/ubuntu/miniconda3/envs/ai_processor/lib/python3.12/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
<function _ConnectionBase.__del__ at 0x7071e4343380>
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/ai_processor/lib/python3.12/multiprocessing/connection.py", line 133, in __del__
    self._close()
  File "/home/ubuntu/miniconda3/envs/ai_processor/lib/python3.12/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/ai_processor/lib/pyt

KeyboardInterrupt: 